In [ ]:
from set_home_directory import get_project_root_homedir_in_sys_path
project_root, main_dir = get_project_root_homedir_in_sys_path("inter_areal_predictability")
if project_root is None:
    raise RuntimeError(f"Project root not found: ensure a folder named '{project_root}' exists in one of the sys.path entries.")
print("Project root found:", project_root)

import sys
import pickle
import time
from matplotlib import collections
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap, Normalize
import os
import json


results_dir = os.path.join(project_root,'results/fig_7/')

# ensure the results directory exists
os.makedirs(results_dir, exist_ok=True)
os.chdir(project_root)

sys.path.insert(0,os.path.join(main_dir,'utils/'))
sys.path.insert(0,main_dir)

from utils.fig_7_functions import make_mouse_df_time, add_stars_2_sets,plot_null_line, make_monkey_df_time
from utils.fig_7_functions import get_refons_refoffs, make_df_timelags
from utils.fig_3_functions import color_label
from utils.stats_functions import get_comparison_test_stars

fig_dir = project_root + '/results/paper_figures/'
save_figs = False


## upload

In [ ]:

colors = ['#ddf2f0', '#55C0B7', '#112624']
norm = Normalize(vmin=0, vmax=1)
v4_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)
colors = ['#FFE8E8', '#F58B8E', '#551312']
v1_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)

mouse_stats_path = os.path.join(project_root, 'results/fig_7',f'mouse_stats.pkl')
with open(mouse_stats_path, 'rb') as f:
	mouse_stats = pickle.load(f)


monkey_names = ['L','A', 'D']
all_monkey_stats = {}
for monkey_name in monkey_names:
    monkey_stats_path = os.path.join(project_root, 'results/fig_7',f'monkey_{monkey_name}_stats.pkl')
    with open(monkey_stats_path, 'rb') as f:
        all_monkey_stats[monkey_name] = pickle.load(f)

## Mouse Shuffle Trial Repeats

### plotting

In [ ]:
df_mouse_all = make_mouse_df_time(mouse_stats)
df_mouse=df_mouse_all[(df_mouse_all['SNR']>2)&(df_mouse_all['split-half r']>0.8)&(df_mouse_all['control_shuffle']==False)]
df_mouse_null=df_mouse_all[(df_mouse_all['SNR']>2)&(df_mouse_all['split-half r']>0.8)&(df_mouse_all['control_shuffle']==True)]
melted_mouse_df = df_mouse.melt(id_vars=['Dataset Type', 'Activity Type', 'Mouse', 'Mouse Name', 'Area',
       'Direction', 'SNR', 'split-half r', 'max r² val', '1-vs-rest r²',
       'control_shuffle', 'Neuron',],
                                        value_vars=['EV', 'EV shuffled'],
                                        var_name='Shuffle Type',
                                        value_name='EV fraction')


In [ ]:
order=['L4→L2/3', 'L2/3→L4']
hue_order = ['EV', 'EV shuffled']
x='Direction'
y='EV fraction'
hue='Shuffle Type'

In [ ]:
# Fig 7B 
palette = ['#72BEB7','#EDAEAE']

fontsize=7
fig, ax = plt.subplots(figsize=(1.5,1.5))
sns.violinplot(data=melted_mouse_df, x=x, y=y, 
            hue=hue, order=order,hue_order=hue_order,
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7, 'markersize':3, 'markeredgewidth':1},
            saturation=1,cut=0)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles=handles, labels=['unshuffled', 'shuffled'], fontsize=6*0.8, loc=(0.55,0.9))
legend.get_frame().set_linewidth(0.2)

color_label(ax, palette,fontsize, x_offset=-0.1, predictor_color='black')
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3)
ax.set_xlabel(None)
ax.set_ylabel(y, fontsize=fontsize, labelpad=1)
sns.despine()

custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)

add_stars_2_sets(melted_mouse_df, neuron_property=y, 
hue=hue, ax=ax, fontsize=fontsize,x=x, x_order=order,height1=0.8, height2=0.8, perm_type='paired', hierarchical=True)
plot_null_line(df_mouse_null, neuron_property='EV', ax=ax)
if save_figs is True:
    plt.savefig(fig_dir + 'fig7b_mouse_shuffle_trial_ev.pdf', transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
df_mouse_ = make_mouse_df_time(mouse_stats).query('control_shuffle==False')[['Dataset Type', 'Mouse', 'Mouse Name', 'Area', 'Direction', 'EV','1-vs-rest r²',
       'Neuron', 'EV shuffled', 'Activity Type']]
df_mouse_spont = make_mouse_df_time(mouse_stats, dataset_types=['ori32_spont','natimg32_spont']).query('control_shuffle==False')[['Dataset Type', 'Mouse', 'Mouse Name', 'Area', 'Direction', 'EV','1-vs-rest r²',
       'Neuron', 'EV shuffled', 'Activity Type']]
df_mouse_spont_ =df_mouse_spont.rename(columns={'EV':'EV gray screen','EV shuffled':'EV shuffled gray screen' })
df_mouse_resp_spont = pd.merge(df_mouse_, df_mouse_spont_, on=['Mouse', 'Mouse Name', 'Area', 'Direction','1-vs-rest r²',
       'Neuron'])
df_mouse_resp_spont_sorted = df_mouse_resp_spont.sort_values('EV gray screen')

In [ ]:
# Fig 7C left
area='L2/3'
palette=v4_cmap

fontsize=7
line_color='black'
x= 'EV'
y='EV shuffled'
hue='EV gray screen'

# Calculate global min and max for consistent normalization
global_min = df_mouse_resp_spont_sorted[hue].min()
global_max = df_mouse_resp_spont_sorted[hue].max()

fig, ax=plt.subplots(figsize=(1.2,1.2))
sns.scatterplot(data=df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area],
x=x, y=y, hue=hue, palette=palette, s=5, hue_norm=(global_min,global_max))

ev_max = df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area][x].max()
ev_shuffled_max= df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area][y].max()
ev_min = df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area][x].min()
ev_shuffled_min = df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area][y].min()




legend = ax.legend(loc=(1.05,0.3),fontsize=fontsize*0.8)
ax.tick_params(axis='both', labelsize=fontsize)
ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=1)
ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=1)
ax.set_aspect('equal')
legend.set_title(title='EV gray screen',prop={'size':fontsize*0.8})
legend.get_frame().set_linewidth(0.2)

ax.plot([0, max(ev_max,ev_shuffled_max)], [0,max(ev_max, ev_shuffled_max)],color=line_color, linestyle='--', linewidth=0.8)
ax.spines[:].set_linewidth(0.5)
sns.despine()
ax.legend_.remove()
ax.set_xticks([0,0.5])
ax.set_yticks([0,0.5])

# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.95)
cbar.set_label('EV gray\nscreen', fontsize=fontsize*.7,y=1.1,ha='center', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 -0.04, pos.width, pos.height])


if save_figs is True:
    plt.savefig(fig_dir + 'fig7_supp_mouse_all_shuffled_vs_unshuffled_l23.pdf', transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
# Fig 7C right
area='L4'
palette=v1_cmap

fontsize=7
line_color='black'
x= 'EV'
y='EV shuffled'
hue='EV gray screen'

fig, ax=plt.subplots(figsize=(1.2,1.2))
sns.scatterplot(data=df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area],
x=x, y=y, hue=hue, palette=palette, s=5, hue_norm=(global_min,global_max))

ev_max = df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area][x].max()
ev_shuffled_max= df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area][y].max()
ev_min = df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area][x].min()
ev_shuffled_min = df_mouse_resp_spont_sorted[df_mouse_resp_spont_sorted.Area==area][y].min()

legend = ax.legend(loc=(1.05,0.3),fontsize=fontsize*0.8)
ax.tick_params(axis='both', labelsize=fontsize)
ax.set_ylabel(None)
ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=1)
ax.set_aspect('equal')
legend.set_title(title='EV gray screen',prop={'size':fontsize*0.8})
legend.get_frame().set_linewidth(0.2)

ax.plot([0, max(ev_max,ev_shuffled_max)], [0,max(ev_max, ev_shuffled_max)],color=line_color, linestyle='--', linewidth=0.8)
ax.spines[:].set_linewidth(0.5)
sns.despine()
ax.legend_.remove()
ax.set_xticks([0,0.5])
ax.set_yticks([0,0.5], labels=[])



# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.95)
cbar.set_label('EV gray\nscreen', fontsize=fontsize*.7,y=1.1,ha='center', rotation=0)  # Customize label as needed
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)

# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 - 0.04, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir + 'fig7_supp_mouse_all_shuffled_vs_unshuffled_l4.pdf', transparent=True, bbox_inches='tight')

plt.show()

## Monkey L Shuffle Trial Repeats

### plotting

In [ ]:

monkey_name =  'L'
monkey_stats = all_monkey_stats[monkey_name]
df_ = make_monkey_df_time(monkey_stats)
df = df_[(df_['SNR']>2)&(df_['split-half r']>0.8)]
melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')

In [ ]:
# Fig 7E
fontsize=7
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1.5,1.5))
sns.violinplot(data=melted_df, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7, 'markersize':3, 'markeredgewidth':1},
            saturation=1,cut=0)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles=handles, labels=['unshuffled', 'shuffled'], fontsize=6*0.8, loc=(0.55,0.95))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=1)
ax.set(xlabel=None)

palette = ['#72BEB7','#EDAEAE']
color_label(ax, palette,fontsize, predictor_color='black')
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)
add_stars_2_sets(melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])], neuron_property='Value',
hue='Analysis Type', ax=ax, fontsize=fontsize, x='Area', x_order=['V4', 'V1'],height1=0.8, height2=0.8, perm_type='paired')
plot_null_line(df, neuron_property='control shuffle EV', ax=ax)
if save_figs is True:
    plt.savefig(fig_dir + 'fig7d_monkey_shuffle_trial_ev.pdf', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
df_ = make_monkey_df_time(monkey_stats)
df_spont=make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont','RF_thin_spont','RF_large_spont'])
df_spont_ =df_spont.rename(columns={'Activity Type':'Activity Type spont','EV':'EV gray screen', 'EV shuffled':'EV shuffled gray screen','Max Corr Vals':'Max Corr Val Spont','null_shuffled': 'null_shuffled spont',
'Coeff Val':'Coeff Val spont'})               
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV gray screen')

In [ ]:
# Fig 7F left
fontsize=7
area='V4'
line_color='black'

hue='EV gray screen'
# Calculate global min and max for consistent normalization
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

fig, ax=plt.subplots(figsize=(1.2,1.2))
sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
x='EV', y='EV shuffled', hue='EV gray screen', palette=v4_cmap, s=5, hue_norm=(global_min,global_max))
ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
legend = ax.legend(loc=(1.05,0.3),fontsize=fontsize*0.8)
ax.tick_params(axis='both', labelsize=fontsize, pad=1)
ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=1)
ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=1)
ax.set_aspect('equal')
legend.set_title(title='EV gray screen',prop={'size':6*0.8})
legend.get_frame().set_linewidth(0.2)
ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
sns.despine()
ax.legend_.remove()
ax.spines[:].set_linewidth(0.5)
ax.set_xticks([0,0.5])
ax.set_yticks([0,0.5])

# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v4_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=fontsize*.7,y=1.1,ha='center', rotation=0)  # Customize label as needed
cbar.ax.set_yticks([0.2,0.4,0.6])
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)
# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 -0.04, pos.width, pos.height])

if save_figs is True:
    plt.savefig(fig_dir + 'fig7_monkey_all_shuffled_vs_unshuffled_V4.pdf', transparent=True, bbox_inches='tight')
plt.show()


In [ ]:
# Fig 7F right
fontsize=7
area='V1'
line_color='black'
fig, ax=plt.subplots(figsize=(1.2,1.2))
sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
x='EV', y='EV shuffled', hue='EV gray screen', palette=v1_cmap, s=5, hue_norm=(global_min,global_max))
ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
legend = ax.legend(loc=(1.05,0.3),fontsize=fontsize*0.8)
ax.tick_params(axis='both', labelsize=fontsize, pad=1)
ax.set_ylabel(None)
ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=1)
ax.set_aspect('equal')
legend.set_title(title='EV gray screen',prop={'size':6*0.8})
legend.get_frame().set_linewidth(0.2)
ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
sns.despine()
ax.legend_.remove()
ax.spines[:].set_linewidth(0.5)
ax.set_xticks([0,0.5])
ax.set_yticks([0,0.5], labels='')
# Create a colorbar with the custom colormap
norm = Normalize(global_min,global_max)
sm = plt.cm.ScalarMappable(cmap=v1_cmap, norm=norm)
sm.set_array([])  # Required for matplotlib to recognize the mappable object
cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
cbar.set_label('EV gray\nscreen', fontsize=fontsize*.7,y=1.1,ha='center', rotation=0)  # Customize label as needed
cbar.ax.set_yticks([0.2,0.4,0.6])
cbar.ax.tick_params(labelsize=4, pad=0.02)
cbar.ax.spines[:].set_linewidth(0.5)
# Adjust the colorbar's vertical position
# Get the original position of the colorbar
pos = cbar.ax.get_position()
cbar.ax.set_position([pos.x0, pos.y0 -0.04, pos.width, pos.height])
if save_figs is True:
    plt.savefig(fig_dir + 'fig7_monkey_all_shuffled_vs_unshuffled_V1.pdf', transparent=True, bbox_inches='tight')
plt.show()


### supplemental

In [ ]:
df_ = make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont','RF_thin_spont','RF_large_spont'])
df = df_[(df_['SNR']>2)&(df_['split-half r']>0.8)]
melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')

In [ ]:
# Fig Supp 12A
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1,1.2))
sns.violinplot(data=melted_df, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7},
            saturation=1,cut=0)
legend = ax.legend(fontsize=fontsize*0.8, loc=(0.15,0.8))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, length=2)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)
add_stars_2_sets(melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])], neuron_property='Value',
hue='Analysis Type', ax=ax, fontsize=fontsize, x='Area', x_order=['V4', 'V1'],height1=0.7, height2=0.7, perm_type='paired')
plot_null_line(df, neuron_property='control shuffle EV', ax=ax)
# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=1.15)
palette = ['#72BEB7','#EDAEAE']
ax.set_ylim(top=0.79)
color_label(ax, palette,fontsize, predictor_color='black', y_offset=-0.105)
if save_figs is True:
    plt.savefig(fig_dir + 'fig7d_monkey_shuffle_trial_spont_ev.pdf', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
df = make_monkey_df_time(monkey_stats)
df_spont_=make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont','RF_thin_spont','RF_large_spont'])
df_ =df.rename(columns={'Activity Type':'Activity Type stimulus','EV':'EV stimulus', 'EV shuffled':'EV shuffled stimulus','Max Corr Vals':'Max Corr Val stim','null_shuffled': 'null_shuffled stim',
'Coeff Val':'Coeff Val stim'})               
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV stimulus')

In [ ]:
# Fig Supp 12E top and bottom
# combined figure for supp
fontsize=6

line_color='black'
fig, axes=plt.subplots(figsize=(0.95,2.2), nrows=2, sharex=True, sharey=True)

hue='EV stimulus'
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue=hue, palette=color_map, s=5, 
 	legend=False, ax=ax)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, size=2)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=0)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=0)
	ax.set_aspect('equal')
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.5])
	ax.set_yticks([0,0.5])
	# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=0.99)

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.9)
	cbar.ax.set_title('EV\nstimulus', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.tick_params(labelsize=4, pad=0.02, width=0.3, size=2)
	cbar.ax.spines[:].set_linewidth(0.5)

	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	cbar.ax.set_position([pos.x0+0.07, pos.y0 - 0.04, pos.width, pos.height])

plt.subplots_adjust(hspace=0.5)
if save_figs is True:
    plt.savefig(fig_dir + 'fig7_supp_monkey_spont_shuffled_vs_unshuffled_both_areas.pdf', transparent=True, bbox_inches='tight')

# plt.show()


## Monkey L timewindows offset

### plotting

In [ ]:
monkey_name = 'L'
monkey_stats = all_monkey_stats[monkey_name]

# Define your custom colormap colors (e.g., dark blue to light blue)
colors = ['white', '#55C0B7', '#054C47']
# Normalize the colormap between 0 and 1 (adjust as needed for your data)
norm = Normalize(vmin=0, vmax=1)
# Create the custom colormap
v4_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)


ref_area='V4'
ref_duration=200
w_size=10
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v4_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)




In [ ]:
colors = ['white', '#F58B8E', '#94221F']
# Create the custom colormap
v1_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)

ref_area='V1'
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v1_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True, w_size=w_size)



In [ ]:
# Fig 7I
fig, axes = plt.subplots(1,2, figsize=(2.4,1.1), sharey=True, sharex=True)

for ax, (ref_area, v_cmap) in zip(axes, zip(['V4','V1'], [v4_cmap, v1_cmap])):
	if ref_area=='V4':
		df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)]
		max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v4.Ref_Times.unique())
	elif ref_area=='V1':
		df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)]
		max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v1.Ref_Times.unique())

	sns.heatmap(data, cmap=v_cmap, vmin=0, vmax=0.8, ax=ax)
	ax.tick_params(axis='both', labelsize=6, pad=1)
	if ref_area=='V4':
		ax.set_ylabel('time window', fontsize=6, labelpad=0)
	else:
		ax.set_ylabel(None)
	ax.set_xlabel('predictor time\noffset (ms)', fontsize=6)

	# Access the colorbar and set fontsize of ticks
	cbar = ax.collections[0].colorbar
	cbar.ax.yaxis.set_tick_params(labelsize=6)
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)
	# ax.set_title(f'{predictor_area}→{ref_area}', fontsize=6)
	# ax.set_xticks
plt.subplots_adjust(wspace=0.4)
if save_figs is True:
	plt.savefig(fig_dir + 'fig7_monkey_timelag_percents_snr_V4_V1_10ms.pdf', transparent=True, bbox_inches='tight')


### supplemental

In [ ]:
monkey_name = 'L'
monkey_stats = all_monkey_stats[monkey_name]

# Define your custom colormap colors (e.g., dark blue to light blue)
colors = ['white', '#55C0B7', '#054C47']
# Normalize the colormap between 0 and 1 (adjust as needed for your data)
norm = Normalize(vmin=0, vmax=1)
# Create the custom colormap
v4_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)


ref_area='V4'
ref_duration=200
w_size=10
condition_type='SNR_spont'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v4_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)

colors = ['white', '#F58B8E', '#94221F']
# Create the custom colormap
v1_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)

ref_area='V1'
condition_type='SNR_spont'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v1_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True, w_size=w_size)



In [ ]:
# Fig Supp 12D top 
fig, axes = plt.subplots(1,2, figsize=(2,0.6), sharey=True, sharex=True)

for ax, (ref_area, v_cmap) in zip(axes, zip(['V4','V1'], [v4_cmap, v1_cmap])):
	if ref_area=='V4':
		df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)]
		max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v4.Ref_Times.unique())
	elif ref_area=='V1':
		df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)]
		max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v1.Ref_Times.unique())

	sns.heatmap(data, cmap=v_cmap, vmin=0, vmax=0.8, ax=ax)
	ax.tick_params(axis='both', labelsize=6, pad=1, size=2, width=0.3)
	if ref_area=='V4':
		ax.set_ylabel('time window', fontsize=6, labelpad=0)
	else:
		ax.set_ylabel(None)
	ax.set_xlabel('predictor time\noffset (ms)', fontsize=6)

	# Access the colorbar and set fontsize of ticks
	cbar = ax.collections[0].colorbar
	cbar.ax.yaxis.set_tick_params(labelsize=6*0.7, size=1.5,width=0.3)
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)
	# ax.set_title(f'{predictor_area}→{ref_area}', fontsize=6)
	# ax.set_xticks
	ax.set_xticklabels([])
	ax.set_xlabel('')
plt.subplots_adjust(wspace=0.3)
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_supp_monkey_{monkey_name}_timelag_percents_snr_spont_V4_V1_10ms.pdf', transparent=True, bbox_inches='tight')


## Monkey A Shuffle Trial Repeats

### plotting

In [ ]:
monkey_name =  'A'
monkey_stats = all_monkey_stats[monkey_name]
df_ = make_monkey_df_time(monkey_stats)
df = df_[(df_['SNR']>2)&(df_['split-half r']>0.8)]
melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')

In [ ]:
# Fig Supp 11A
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1.1,1.5))
sns.violinplot(data=melted_df, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7, 'markersize':3, 'markeredgewidth':1},
            saturation=1,cut=0)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles=handles, labels=['unshuffled', 'shuffled'], fontsize=6*0.8, loc=(0.3,0.92))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)

palette = ['#72BEB7','#EDAEAE']
color_label(ax, palette,fontsize, predictor_color='black')
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)
add_stars_2_sets(melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])], neuron_property='Value',
hue='Analysis Type', ax=ax, fontsize=fontsize, x='Area', x_order=['V4', 'V1'],height1=0.8, height2=0.8, perm_type='paired')
plot_null_line(df, neuron_property='control shuffle EV', ax=ax)
ax.set_ylim(top=0.77)
if save_figs is True:
    plt.savefig(fig_dir + f'fig7d_monkey_{monkey_name}_shuffle_trial_ev.pdf', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
df_ = make_monkey_df_time(monkey_stats)
df_spont=make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont','RF_thin_spont','RF_large_spont'])
df_spont_ =df_spont.rename(columns={'Activity Type':'Activity Type spont','EV':'EV gray screen', 'EV shuffled':'EV shuffled gray screen','Max Corr Vals':'Max Corr Val Spont','null_shuffled': 'null_shuffled spont',
'Coeff Val':'Coeff Val spont'})               
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV gray screen')

In [ ]:
# Fig Supp 11F left and right
fontsize=6

line_color='black'

hue='EV gray screen'
# Calculate global min and max for consistent normalization
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

fig, axes=plt.subplots(ncols=2, figsize=(2.5,1.2), sharex=True, sharey=True)

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue='EV gray screen', palette=color_map, 
	s=5, hue_norm=(global_min,global_max), ax=ax, legend=False)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	# legend = ax.legend(loc=(1.05,0.3),fontsize=fontsize*0.8)
	ax.tick_params(axis='both', labelsize=fontsize, pad=1)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=1)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=1)
	ax.set_aspect('equal')
	# legend.set_title(title='EV gray screen',prop={'size':6*0.8})
	legend.get_frame().set_linewidth(0.2)
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.5])
	ax.set_yticks([0,0.5])

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.75)
	# cbar.set_label('EV gray\nscreen', fontsize=fontsize*.8,
    #             	y=1.25,ha='right', rotation=0)  # Customize label as needed
	cbar.ax.set_title('EV gray\nscreen', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.set_yticks([0.2,0.4,0.6])
	cbar.ax.tick_params(labelsize=4, pad=0.02)
	cbar.ax.spines[:].set_linewidth(0.5)
	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	cbar.ax.set_position([pos.x0, pos.y0 -0.08, pos.width, pos.height])
	
plt.subplots_adjust(wspace=0.14)
if save_figs is True:
    plt.savefig(fig_dir + f'fig7_monkey_{monkey_name}_all_shuffled_vs_unshuffled_both_areas.pdf', transparent=True, bbox_inches='tight')
plt.show()


### supplemental

In [ ]:
df_ = make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont','RF_thin_spont','RF_large_spont'])
df = df_[(df_['SNR']>2)&(df_['split-half r']>0.8)]
melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')

In [ ]:
# Fig Supp 12B left
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1,1.2))
sns.violinplot(data=melted_df, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7},
            saturation=1,cut=0)
legend = ax.legend(fontsize=fontsize*0.8, loc=(0.3,0.93))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, length=2)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)
add_stars_2_sets(melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])], neuron_property='Value',
hue='Analysis Type', ax=ax, fontsize=fontsize, x='Area', x_order=['V4', 'V1'],height1=0.8, height2=0.8, perm_type='paired')
plot_null_line(df, neuron_property='control shuffle EV', ax=ax)
# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=1.15)
palette = ['#72BEB7','#EDAEAE']
ax.legend_.remove()
ax.set_ylim(top=0.69)
color_label(ax, palette,fontsize, predictor_color='black', y_offset=-0.105)
if save_figs is True:
    plt.savefig(fig_dir + f'fig7d_monkey_{monkey_name}_shuffle_trial_spont_ev.pdf', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
df = make_monkey_df_time(monkey_stats)
df_spont_=make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont','RF_thin_spont','RF_large_spont'])
df_ =df.rename(columns={'Activity Type':'Activity Type stimulus','EV':'EV stimulus', 'EV shuffled':'EV shuffled stimulus','Max Corr Vals':'Max Corr Val stim','null_shuffled': 'null_shuffled stim',
'Coeff Val':'Coeff Val stim'})               
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV stimulus')

In [ ]:
#Fig Supp 12F top left and right
# combined figure for supp
fontsize=6

line_color='black'
fig, axes=plt.subplots(figsize=(2,1.1), ncols=2, sharex=True, sharey=True)

hue='EV stimulus'
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue=hue, palette=color_map, s=5, 
 	legend=False, ax=ax)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, size=2)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=0)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=0)
	ax.set_aspect('equal')
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.5])
	ax.set_yticks([0,0.5])
	# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=0.99)

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.75)
	cbar.ax.set_title('EV\nstimulus', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.tick_params(labelsize=4, pad=0.02, width=0.3, size=2)
	cbar.ax.spines[:].set_linewidth(0.5)

	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	cbar.ax.set_position([pos.x0+0.07, pos.y0 - 0.04, pos.width, pos.height])
	ax.set_xticklabels([])
	ax.set_xlabel(None)
	ax.set_xlim(left=-0.02, right=0.64)
plt.subplots_adjust(wspace=0.14)
if save_figs is True:
    plt.savefig(fig_dir + f'fig7_supp_monkey_{monkey_name}_spont_shuffled_vs_unshuffled_both_areas.pdf', transparent=True, bbox_inches='tight')

# plt.show()


## Monkey A timewindows offset

### plotting

In [ ]:
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]

# Define your custom colormap colors (e.g., dark blue to light blue)
colors = ['white', '#55C0B7', '#054C47']
# Normalize the colormap between 0 and 1 (adjust as needed for your data)
norm = Normalize(vmin=0, vmax=1)
# Create the custom colormap
v4_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)


ref_area='V4'
ref_duration=200
w_size=10
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v4_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)



colors = ['white', '#F58B8E', '#94221F']
# Create the custom colormap
v1_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)

ref_area='V1'
condition_type='SNR'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v1_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True, w_size=w_size)


In [ ]:
# Fig Supp 11E top
fig, axes = plt.subplots(1,2, figsize=(2.2,0.9), sharey=True, sharex=True)

for ax, (ref_area, v_cmap) in zip(axes, zip(['V4','V1'], [v4_cmap, v1_cmap])):
	if ref_area=='V4':
		df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)]
		max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v4.Ref_Times.unique())
	elif ref_area=='V1':
		df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)]
		max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v1.Ref_Times.unique())

	sns.heatmap(data, cmap=v_cmap, vmin=0, vmax=0.8, ax=ax)
	ax.tick_params(axis='both', labelsize=6, pad=1, size=2.5, width=0.5)
	if ref_area=='V4':
		ax.set_ylabel('time window', fontsize=6, labelpad=0)
	else:
		ax.set_ylabel(None)
	ax.set_xlabel('predictor time\noffset (ms)', fontsize=6)

	# Access the colorbar and set fontsize of ticks
	cbar = ax.collections[0].colorbar
	cbar.ax.yaxis.set_tick_params(labelsize=6*0.7, size=1.5,width=0.3)
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)
	# ax.set_title(f'{predictor_area}→{ref_area}', fontsize=6)
	# ax.set_xticks
	ax.set_xticklabels([])
	ax.set_xlabel('')
plt.subplots_adjust(wspace=0.4)
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_monkey_{monkey_name}_timelag_percents_snr_V4_V1_10ms.pdf', transparent=True, bbox_inches='tight')


### supplemental

In [ ]:
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]

# Define your custom colormap colors (e.g., dark blue to light blue)
colors = ['white', '#55C0B7', '#054C47']
# Normalize the colormap between 0 and 1 (adjust as needed for your data)
norm = Normalize(vmin=0, vmax=1)
# Create the custom colormap
v4_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)


ref_area='V4'
ref_duration=200
w_size=10
condition_type='SNR_spont'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v4_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)




colors = ['white', '#F58B8E', '#94221F']
# Create the custom colormap
v1_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)

ref_area='V1'
condition_type='SNR_spont'
ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
df_v1_ = make_df_timelags(monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True, w_size=w_size)



In [ ]:
# Fig Supp 12D middle
fig, axes = plt.subplots(1,2, figsize=(2,0.6), sharey=True, sharex=True)

for ax, (ref_area, v_cmap) in zip(axes, zip(['V4','V1'], [v4_cmap, v1_cmap])):
	if ref_area=='V4':
		df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)]
		max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v4.Ref_Times.unique())
	elif ref_area=='V1':
		df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)]
		max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
		heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
		data =heatmap_data_max/heatmap_data_all
		data = data.reindex(df_v1.Ref_Times.unique())

	sns.heatmap(data, cmap=v_cmap, vmin=0, vmax=0.8, ax=ax)
	ax.tick_params(axis='both', labelsize=6, pad=1, size=2, width=0.3)
	if ref_area=='V4':
		ax.set_ylabel('time window', fontsize=6, labelpad=0)
	else:
		ax.set_ylabel(None)
	ax.set_xlabel('predictor time\noffset (ms)', fontsize=6)

	# Access the colorbar and set fontsize of ticks
	cbar = ax.collections[0].colorbar
	cbar.ax.yaxis.set_tick_params(labelsize=6*0.7, size=1.5,width=0.3)
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)
	# ax.set_title(f'{predictor_area}→{ref_area}', fontsize=6)
	# ax.set_xticks
	ax.set_xticklabels([])
	ax.set_xlabel('')
plt.subplots_adjust(wspace=0.3)
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_supp_monkey_{monkey_name}_timelag_percents_snr_spont_V4_V1_10ms.pdf', transparent=True, bbox_inches='tight')


## Monkey D Shuffle Trial Repeats

### plotting

In [ ]:
monkey_name =  'D'
monkey_stats = all_monkey_stats[monkey_name]
df_ = make_monkey_df_time(monkey_stats, dataset_types=['SNR'])
df = df_[(df_['SNR']>2)&(df_['split-half r']>0.6)]
melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')

In [ ]:
# Fig Supp 11C
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1.1,1.5))
sns.violinplot(data=melted_df, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7, 'markersize':3, 'markeredgewidth':1},
            saturation=1,cut=0)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles=handles, labels=['unshuffled', 'shuffled'], fontsize=6*0.8, loc=(0.3,0.92))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)
palette = ['#72BEB7','#EDAEAE']
color_label(ax, palette,fontsize, predictor_color='black', y_offset=-0.095)
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)
add_stars_2_sets(melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])], neuron_property='Value',
hue='Analysis Type', ax=ax, fontsize=fontsize, x='Area', x_order=['V4', 'V1'],height1=0.5, height2=0.5, perm_type='paired')
plot_null_line(df, neuron_property='control shuffle EV', ax=ax)
ax.set_ylim(top=0.68, bottom=-0.01)
ax.legend_.remove()
if save_figs is True:
    plt.savefig(fig_dir + f'fig7d_monkey_{monkey_name}_shuffle_trial_ev.pdf', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
df_ = make_monkey_df_time(monkey_stats, dataset_types=['SNR'])
df_spont=make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont'])
df_spont_ =df_spont.rename(columns={'Activity Type':'Activity Type spont','EV':'EV gray screen', 'EV shuffled':'EV shuffled gray screen','Max Corr Vals':'Max Corr Val Spont','null_shuffled': 'null_shuffled spont',
'Coeff Val':'Coeff Val spont'})               
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV gray screen')

In [ ]:
# Fig Supp 11G left and right
fontsize=6

line_color='black'

hue='EV gray screen'
# Calculate global min and max for consistent normalization
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

fig, axes=plt.subplots(ncols=2, figsize=(2.5,1.2), sharex=True, sharey=True)

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue='EV gray screen', palette=color_map, 
	s=5, hue_norm=(global_min,global_max), ax=ax, legend=False)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	# legend = ax.legend(loc=(1.05,0.3),fontsize=fontsize*0.8)
	ax.tick_params(axis='both', labelsize=fontsize, pad=1)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=1)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=1)
	ax.set_aspect('equal')
	# legend.set_title(title='EV gray screen',prop={'size':6*0.8})
	legend.get_frame().set_linewidth(0.2)
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.2])
	ax.set_yticks([0,0.2])

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.75)
	# cbar.set_label('EV gray\nscreen', fontsize=fontsize*.8,
    #             	y=1.25,ha='right', rotation=0)  # Customize label as needed
	cbar.ax.set_title('EV gray\nscreen', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.set_yticks([0,0.1])
	cbar.ax.tick_params(labelsize=4, pad=0.02)
	cbar.ax.spines[:].set_linewidth(0.5)
	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	cbar.ax.set_position([pos.x0, pos.y0 -0.08, pos.width, pos.height])
plt.subplots_adjust(wspace=0.14)
if save_figs is True:
    plt.savefig(fig_dir + f'fig7_monkey_{monkey_name}_all_shuffled_vs_unshuffled_both_areas.pdf', transparent=True, bbox_inches='tight')
plt.show()


### supplemental

In [ ]:
monkey_name =  'D'
monkey_stats = all_monkey_stats[monkey_name]
df_ = make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont'])
df = df_[(df_['SNR']>2)&(df_['split-half r']>0.6)]
melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')

In [ ]:
# Fig Supp 12C left
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1,1.2))
sns.violinplot(data=melted_df, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7},
            saturation=1,cut=0)
legend = ax.legend(fontsize=fontsize*0.8, loc=(0.55,0.9))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, size=2)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)
add_stars_2_sets(melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])], neuron_property='Value',
hue='Analysis Type', ax=ax, fontsize=fontsize, x='Area', x_order=['V4', 'V1'],height1=0.35, height2=0.35, perm_type='paired')
plot_null_line(df, neuron_property='control shuffle EV', ax=ax)
# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=1.15)
palette = ['#72BEB7','#EDAEAE']
color_label(ax, palette,fontsize, predictor_color='black', y_offset=-0.075)
# ax.set_yticks(ticks=[0,0.5])
ax.set_ylim(top=0.49, bottom=-0.01)
ax.legend_.remove()
if save_figs is True:
    plt.savefig(fig_dir + f'fig7d_monkey_{monkey_name}_shuffle_trial_spont_ev.pdf', transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
df = make_monkey_df_time(monkey_stats, ['SNR'])
df_spont_=make_monkey_df_time(monkey_stats, dataset_types=['SNR_spont'])
df_ =df.rename(columns={'Activity Type':'Activity Type stimulus','EV':'EV stimulus', 'EV shuffled':'EV shuffled stimulus','Max Corr Vals':'Max Corr Val stim','null_shuffled': 'null_shuffled stim',
'Coeff Val':'Coeff Val stim'})               
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV stimulus')

In [ ]:
#Fig Supp 12G top left and right
# combined figure for supp
fontsize=6

line_color='black'
fig, axes=plt.subplots(figsize=(2,1.1), ncols=2, sharex=True, sharey=True)

hue='EV stimulus'
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue=hue, palette=color_map, s=5, 
 	legend=False, ax=ax)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, size=2)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=0)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=0)
	ax.set_aspect('equal')
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.4])
	ax.set_yticks([0,0.4])
	# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=0.99)

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.75)
	cbar.ax.set_title('EV\nstimulus', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.tick_params(labelsize=4, pad=0.02, width=0.3, size=2)
	cbar.ax.spines[:].set_linewidth(0.5)

	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	cbar.ax.set_position([pos.x0+0.07, pos.y0 - 0.04, pos.width, pos.height])
	ax.set_xticklabels([])
	ax.set_xlabel(None)
	ax.set_xlim(left=-0.02, right=0.41)
plt.subplots_adjust(wspace=0.14)
if save_figs is True:
    plt.savefig(fig_dir + f'fig7_supp_monkey_{monkey_name}_spont_shuffled_vs_unshuffled_both_areas.pdf', transparent=True, bbox_inches='tight')

# plt.show()


## Monkey $L_{A}$ timewindow offset 

In [ ]:

# Define your custom colormap colors (e.g., dark blue to light blue)
colors = ['white', '#55C0B7', '#054C47']
# Normalize the colormap between 0 and 1 (adjust as needed for your data)
norm = Normalize(vmin=0, vmax=1)
# Create the custom colormap
v4_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)

colors = ['white', '#F58B8E', '#94221F']
# Create the custom colormap
v1_cmap = LinearSegmentedColormap.from_list('custom_colormap', colors, N=256)
ref_duration=200
w_size=10
condition_type='SNR'

with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'A'

# we are going to loop through all seeds, and collect the variable data per area. after that we will average across seeds.
data_v4_all_seeds = []
data_v1_all_seeds = []

for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	ref_area='V4'
	ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
	df_v4_ = make_df_timelags(subsampled_monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)
	
	ref_area='V1'
	ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
	df_v1_ = make_df_timelags(subsampled_monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True, w_size=w_size)
	for ref_area, v_cmap in zip(['V4','V1'], [v4_cmap, v1_cmap]):
		if ref_area=='V4':
			df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)]
			max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
			heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
			heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
			data =heatmap_data_max/heatmap_data_all
			data = data.reindex(df_v4.Ref_Times.unique())
			data_v4_all_seeds.append(data)
		elif ref_area=='V1':
			df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)]
			max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
			heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
			heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
			data =heatmap_data_max/heatmap_data_all
			data = data.reindex(df_v1.Ref_Times.unique())
			data_v1_all_seeds.append(data)

data_v4_mean = pd.concat(data_v4_all_seeds).groupby(level=0).mean()
data_v1_mean = pd.concat(data_v1_all_seeds).groupby(level=0).mean()
	

In [ ]:
# Fig Supp 11E bottom

fig, axes = plt.subplots(1,2, figsize=(2.2,0.9), sharey=True, sharex=True)
for ax, (ref_area, v_cmap) in zip(axes, zip(['V4','V1'], [v4_cmap, v1_cmap])):
	if ref_area=='V4':
		data = data_v4_mean.reindex(df_v4.Ref_Times.unique())
	elif ref_area=='V1':
		data = data_v1_mean.reindex(df_v1.Ref_Times.unique())

	sns.heatmap(data, cmap=v_cmap, vmin=0, vmax=0.8, ax=ax)
	ax.tick_params(axis='both', labelsize=6, pad=1, size=2.5, width=0.5)
	if ref_area=='V4':
		ax.set_ylabel('time window', fontsize=6, labelpad=0)
	else:
		ax.set_ylabel(None)
	ax.set_xlabel('predictor time\noffset (ms)', fontsize=6)

	# Access the colorbar and set fontsize of ticks
	cbar = ax.collections[0].colorbar
	cbar.ax.yaxis.set_tick_params(labelsize=6*0.7, size=1.5,width=0.3)
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)
	# ax.set_title(f'{predictor_area}→{ref_area}', fontsize=6)
	# ax.set_xticks	
plt.subplots_adjust(wspace=0.4)
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_timelag_percents_snr_V4_V1_10ms.pdf', transparent=True, bbox_inches='tight')


### gray screen

In [ ]:
# we are going to loop through all seeds, and collect the variable data per area. after that we will average across seeds.
data_v4_all_seeds = []
data_v1_all_seeds = []
condition_type='SNR_spont'
for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	ref_area='V4'
	ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
	df_v4_ = make_df_timelags(subsampled_monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True,w_size=w_size)
	
	ref_area='V1'
	ref_ons, ref_offs = get_refons_refoffs(ref_duration, w_size, condition_type)
	df_v1_ = make_df_timelags(subsampled_monkey_stats, condition_type, ref_area, ref_ons,ref_offs, ref_duration, control_neurons=True, w_size=w_size)

	for ref_area, v_cmap in zip(['V4','V1'], [v4_cmap, v1_cmap]):
		if ref_area=='V4':
			df_v4 = df_v4_[(df_v4_['Offset(ms)']<=105)&(df_v4_['Offset(ms)']>=-105)]
			max_value_indices = df_v4.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
			heatmap_data_max = df_v4.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
			heatmap_data_all = df_v4.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
			data =heatmap_data_max/heatmap_data_all
			data = data.reindex(df_v4.Ref_Times.unique())
			data_v4_all_seeds.append(data)
		elif ref_area=='V1':
			df_v1 = df_v1_[(df_v1_['Offset(ms)']<=105)&(df_v1_['Offset(ms)']>=-105)]
			max_value_indices = df_v1.groupby(['Ref_Times','SNR','split-half r','Permutation'])['EV'].idxmax()
			heatmap_data_max = df_v1.loc[max_value_indices].pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
			heatmap_data_all = df_v1.pivot_table(index='Ref_Times', columns='Offset(ms)', values='EV', aggfunc='count')
			data =heatmap_data_max/heatmap_data_all
			data = data.reindex(df_v1.Ref_Times.unique())
			data_v1_all_seeds.append(data)

data_v4_mean = pd.concat(data_v4_all_seeds).groupby(level=0).mean()
data_v1_mean = pd.concat(data_v1_all_seeds).groupby(level=0).mean()

In [ ]:
# Fig Supp 12D bottom
fig, axes = plt.subplots(1,2, figsize=(2,0.6), sharey=True, sharex=True)
for ax, (ref_area, v_cmap) in zip(axes, zip(['V4','V1'], [v4_cmap, v1_cmap])):
	if ref_area=='V4':
		data = data_v4_mean.reindex(df_v4.Ref_Times.unique())
	elif ref_area=='V1':
		data = data_v1_mean.reindex(df_v1.Ref_Times.unique())

	sns.heatmap(data, cmap=v_cmap, vmin=0, vmax=0.8, ax=ax)
	ax.tick_params(axis='both', labelsize=6, pad=1, size=2, width=0.3)
	if ref_area=='V4':
		ax.set_ylabel('time window', fontsize=6, labelpad=0)
	else:
		ax.set_ylabel(None)
	ax.set_xlabel('predictor time\noffset (ms)', fontsize=6)

	# Access the colorbar and set fontsize of ticks
	cbar = ax.collections[0].colorbar
	cbar.ax.yaxis.set_tick_params(labelsize=6*0.7, size=1.5,width=0.3)
	if ref_area=='V4':
		predictor_area='V1'
	elif ref_area=='V1':
		predictor_area='V4'
	cols = list(data.columns)
	if 0 in cols:
		x0 = cols.index(0) + 0.5   # center of the 0-ms bin
		ax.axvline(x0, linestyle='--', linewidth=0.5, color='k', alpha=0.5, zorder=5)

plt.subplots_adjust(wspace=0.3)
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_timelag_percents_snr_spont_V4_V1_10ms.pdf', transparent=True, bbox_inches='tight')

## Monkey $L_{A}$ Shuffle Trial Repeats

### plotting

In [ ]:
x_order=['V4', 'V1']
x ='Area'
hue='Analysis Type'
neuron_property='Value'
perm_type='paired'
perm_t=True
hierarchical=False
mouse_or_date='Date'


with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
main_monkey_name = 'L'
subsample_monkey_name = 'A'

melted_df_monkey_subsample = pd.DataFrame()
pval_dict_monkey_subsample = {}
pval_dict_monkey_subsample['V4'] = []
pval_dict_monkey_subsample['V1'] = []


for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	stim_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' not in k and 'RS' not in k]
	df_ = make_monkey_df_time(subsampled_monkey_stats, dataset_types=stim_datataset_types_)
	df = df_[(df_['SNR']>2)&(df_['split-half r']>0.8)]
	melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')
	melted_df['Subsample Seed'] = seed
	melted_df_monkey_subsample = pd.concat([melted_df_monkey_subsample, melted_df], ignore_index=True)
	df_stats = melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])]
	p_val1, stars1 = get_comparison_test_stars(df_stats[df_stats[x]==x_order[0]], hue, neuron_property, perm_t=perm_t, perm_type=perm_type, 
                                            hierarchical=hierarchical, mouse_or_date=mouse_or_date, central_tendency='median',
                                        	return_pval=True)
	p_val2, stars2 = get_comparison_test_stars(df_stats[df_stats[x]==x_order[1]], hue, neuron_property, perm_t=perm_t, perm_type=perm_type,
                                            hierarchical=hierarchical, mouse_or_date=mouse_or_date, central_tendency='median',
                                            return_pval=True)
	pval_dict_monkey_subsample[x_order[0]].append(p_val1)
	pval_dict_monkey_subsample[x_order[1]].append(p_val2)
	

In [ ]:
# Fig Supp 11B
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1.1,1.5))
sns.violinplot(data=melted_df_monkey_subsample, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7, 'markersize':3, 'markeredgewidth':1},
            saturation=1,cut=0)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles=handles, labels=['unshuffled', 'shuffled'], fontsize=6*0.8, loc=(0.3,0.92))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)

palette = ['#72BEB7','#EDAEAE']
color_label(ax, palette,fontsize, predictor_color='black')
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)
plot_null_line(df, neuron_property='control shuffle EV', ax=ax)

import numpy as np
for x_i in x_order:
	y = melted_df_monkey_subsample[melted_df_monkey_subsample[x]==x_i][neuron_property].max()*0.8
	x_index = x_order.index(x_i)
	p_vals = pval_dict_monkey_subsample[x_i]
	median_p_val = np.median(p_vals)
	if median_p_val < 0.05:
		if median_p_val < 0.001:
			stars = '***'
		elif median_p_val < 0.01:
			stars = '**'
		else:
			stars = '*'
		ax.text(x_index, y, stars, ha='center', va='bottom', fontsize=fontsize)
	else:
		ax.text(x_index, y, 'n.s.', ha='center', va='bottom', fontsize=fontsize*0.8, color='gray')

ax.set_ylim(top=0.77)
ax.legend_.remove()
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_EV_comparison.pdf', transparent=True, bbox_inches='tight')

In [ ]:
import random 
random.seed(17)
subsample_seed = random.choice(subsample_seeds)
with open(os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{subsample_seed}_stats.pkl'), 'rb') as f:
	subsampled_monkey_stats = pickle.load(f)
stim_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' not in k and 'RS' not in k]
spont_dataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and ('spont' in k or 'RS' in k)]
df_ = make_monkey_df_time(subsampled_monkey_stats, dataset_types=stim_datataset_types_)
df_spont = make_monkey_df_time(subsampled_monkey_stats, dataset_types=spont_dataset_types_)
df_spont_ =df_spont.rename(columns={'Activity Type':'Activity Type spont','EV':'EV gray screen', 'EV shuffled':'EV shuffled gray screen','Max Corr Vals':'Max Corr Val Spont','null_shuffled': 'null_shuffled spont',
                                    'Coeff Val':'Coeff Val spont'})   
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV gray screen')	

In [ ]:
# Fig Supp 11H left and right

fontsize=6

line_color='black'

hue='EV gray screen'
# Calculate global min and max for consistent normalization
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

fig, axes=plt.subplots(ncols=2, figsize=(2.5,1.2), sharex=True, sharey=True)

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue='EV gray screen', palette=color_map, 
	s=5, hue_norm=(global_min,global_max), ax=ax, legend=False)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	ax.tick_params(axis='both', labelsize=fontsize, pad=1)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=1)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=1)
	ax.set_aspect('equal')
	legend.get_frame().set_linewidth(0.2)
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.5])
	ax.set_yticks([0,0.5])

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.75)
	cbar.ax.set_title('EV gray\nscreen', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.set_yticks([0.2,0.4,0.6])
	cbar.ax.tick_params(labelsize=4, pad=0.02)
	cbar.ax.spines[:].set_linewidth(0.5)
	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	cbar.ax.set_position([pos.x0, pos.y0 -0.08, pos.width, pos.height])
plt.subplots_adjust(wspace=0.14)
if save_figs is True:
    plt.savefig(fig_dir + f'fig7_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_EV_vs_EVshuffled_colored_by_EVspon.pdf', transparent=True, bbox_inches='tight')
plt.show()


### supplemental

In [ ]:

x_order=['V4', 'V1']
x ='Area'
hue='Analysis Type'
neuron_property='Value'
perm_type='paired'
perm_t=True
hierarchical=False
mouse_or_date='Date'


with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
main_monkey_name = 'L'
subsample_monkey_name = 'A'

melted_df_monkey_subsample = pd.DataFrame()
pval_dict_monkey_subsample = {}
pval_dict_monkey_subsample['V4'] = []
pval_dict_monkey_subsample['V1'] = []


for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	spont_dataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and ('spont' in k or 'RS' in k)]
	df_ = make_monkey_df_time(subsampled_monkey_stats, dataset_types=spont_dataset_types_)
	df = df_[(df_['SNR']>2)&(df_['split-half r']>0.8)]
	melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')
	melted_df['Subsample Seed'] = seed
	melted_df_monkey_subsample = pd.concat([melted_df_monkey_subsample, melted_df], ignore_index=True)
	df_stats = melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])]
	p_val1, stars1 = get_comparison_test_stars(df_stats[df_stats[x]==x_order[0]], hue, neuron_property, perm_t=perm_t, perm_type=perm_type, 
                                            hierarchical=hierarchical, mouse_or_date=mouse_or_date, central_tendency='median',
                                        	return_pval=True)
	p_val2, stars2 = get_comparison_test_stars(df_stats[df_stats[x]==x_order[1]], hue, neuron_property, perm_t=perm_t, perm_type=perm_type,
                                            hierarchical=hierarchical, mouse_or_date=mouse_or_date, central_tendency='median',
                                            return_pval=True)
	pval_dict_monkey_subsample[x_order[0]].append(p_val1)
	pval_dict_monkey_subsample[x_order[1]].append(p_val2)
	

In [ ]:
# Fig Supp 12B right
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1,1.2))
sns.violinplot(data=melted_df_monkey_subsample, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7, 'markersize':3, 'markeredgewidth':1},
            saturation=1,cut=0)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles=handles, labels=['unshuffled', 'shuffled'], fontsize=6*0.8, loc=(0.3,0.92))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)

palette = ['#72BEB7','#EDAEAE']
color_label(ax, palette,fontsize, predictor_color='black', y_offset=-0.105)
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, length=2)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)
plot_null_line(df, neuron_property='control shuffle EV', ax=ax)

import numpy as np
for x_i in x_order:
	y = melted_df_monkey_subsample[melted_df_monkey_subsample[x]==x_i][neuron_property].max()*0.8
	x_index = x_order.index(x_i)
	p_vals = pval_dict_monkey_subsample[x_i]
	median_p_val = np.median(p_vals)
	if median_p_val < 0.05:
		if median_p_val < 0.001:
			stars = '***'
		elif median_p_val < 0.01:
			stars = '**'
		else:
			stars = '*'
		ax.text(x_index, y, stars, ha='center', va='bottom', fontsize=fontsize)
	else:
		ax.text(x_index, y, 'n.s.', ha='center', va='bottom', fontsize=fontsize*0.8, color='gray')
	
ax.set_ylim(top=0.69)
ax.set_ylabel(None)
ax.set_yticklabels([])
ax.legend_.remove()
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_supp_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_EV_comparison_spont.pdf', transparent=True, bbox_inches='tight')

In [ ]:
import random 
random.seed(17)
subsample_seed = random.choice(subsample_seeds)
with open(os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{subsample_seed}_stats.pkl'), 'rb') as f:
	subsampled_monkey_stats = pickle.load(f)
stim_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' not in k and 'RS' not in k]
spont_dataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and ('spont' in k or 'RS' in k)]
df = make_monkey_df_time(subsampled_monkey_stats, dataset_types=stim_datataset_types_)
df_spont_ = make_monkey_df_time(subsampled_monkey_stats, dataset_types=spont_dataset_types_)
df_ =df.rename(columns={'Activity Type':'Activity Type stimulus','EV':'EV stimulus', 'EV shuffled':'EV shuffled stimulus','Max Corr Vals':'Max Corr Val stim','null_shuffled': 'null_shuffled stim',
'Coeff Val':'Coeff Val stim'})               
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV stimulus')

In [ ]:
# Fig Supp 12F bottom left and right
# combined figure for supp
fontsize=6

line_color='black'
fig, axes=plt.subplots(figsize=(2,1.1), ncols=2, sharex=True, sharey=True)

hue='EV stimulus'
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue=hue, palette=color_map, s=5, 
 	legend=False, ax=ax)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, size=2)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=0)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=0)
	ax.set_aspect('equal')
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.5])
	ax.set_yticks([0,0.5])
	# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=0.99)

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.75)
	cbar.ax.set_title('EV\nstimulus', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.tick_params(labelsize=4, pad=0.02, width=0.3, size=2)
	cbar.ax.spines[:].set_linewidth(0.5)

	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	cbar.ax.set_position([pos.x0+0.07, pos.y0 - 0.04, pos.width, pos.height])
	# ax.set_xticklabels([])
	# ax.set_xlabel(None)
	ax.set_xlim(left=-0.02, right=0.64)
plt.subplots_adjust(wspace=0.14)
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_supp_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_EV_vs_EVshuffled_colored_by_EVstim.pdf', transparent=True, bbox_inches='tight')


## Monkey $L_{D}$ subsampled trial repeat shuffle

In [ ]:

x_order=['V4', 'V1']
x ='Area'
hue='Analysis Type'
neuron_property='Value'
perm_type='paired'
perm_t=True
hierarchical=False
mouse_or_date='Date'


with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
main_monkey_name = 'L'
subsample_monkey_name = 'D'

melted_df_monkey_subsample = pd.DataFrame()
pval_dict_monkey_subsample = {}
pval_dict_monkey_subsample['V4'] = []
pval_dict_monkey_subsample['V1'] = []


for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	stim_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' not in k and 'RS' not in k]
	df_ = make_monkey_df_time(subsampled_monkey_stats, dataset_types=stim_datataset_types_)
	df = df_[(df_['SNR']>2)&(df_['split-half r']>0.8)]
	melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')
	melted_df['Subsample Seed'] = seed
	melted_df_monkey_subsample = pd.concat([melted_df_monkey_subsample, melted_df], ignore_index=True)
	df_stats = melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])]
	p_val1, stars1 = get_comparison_test_stars(df_stats[df_stats[x]==x_order[0]], hue, neuron_property, perm_t=perm_t, perm_type=perm_type, 
                                            hierarchical=hierarchical, mouse_or_date=mouse_or_date, central_tendency='median',
                                        	return_pval=True)
	p_val2, stars2 = get_comparison_test_stars(df_stats[df_stats[x]==x_order[1]], hue, neuron_property, perm_t=perm_t, perm_type=perm_type,
                                            hierarchical=hierarchical, mouse_or_date=mouse_or_date, central_tendency='median',
                                            return_pval=True)
	pval_dict_monkey_subsample[x_order[0]].append(p_val1)
	pval_dict_monkey_subsample[x_order[1]].append(p_val2)
	

In [ ]:
# Fig supp 11D
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1.1,1.5))
sns.violinplot(data=melted_df_monkey_subsample, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7, 'markersize':3, 'markeredgewidth':1},
            saturation=1,cut=0)
handles, labels = ax.get_legend_handles_labels()
legend = ax.legend(handles=handles, labels=['unshuffled', 'shuffled'], fontsize=6*0.8, loc=(0.3,0.92))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)
palette = ['#72BEB7','#EDAEAE']
color_label(ax, palette,fontsize, predictor_color='black', y_offset=-0.091)
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)

plot_null_line(df, neuron_property='control shuffle EV', ax=ax)
ax.set_ylim(top=0.68, bottom=-0.01)
ax.legend_.remove()
import numpy as np
for x_i in x_order:
	y = melted_df_monkey_subsample[melted_df_monkey_subsample[x]==x_i][neuron_property].max()*0.8
	x_index = x_order.index(x_i)
	p_vals = pval_dict_monkey_subsample[x_i]
	median_p_val = np.median(p_vals)
	if median_p_val < 0.05:
		if median_p_val < 0.001:
			stars = '***'
		elif median_p_val < 0.01:
			stars = '**'
		else:
			stars = '*'
		ax.text(x_index, y, stars, ha='center', va='bottom', fontsize=fontsize)
	else:
		ax.text(x_index, y, 'n.s.', ha='center', va='bottom', fontsize=fontsize*0.8, color='gray')
if save_figs is True:
	plt.savefig(fig_dir + f'fig7_supp_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_EV_comparison.pdf', transparent=True, bbox_inches='tight')

In [ ]:
import random 
random.seed(17)
subsample_seed = random.choice(subsample_seeds)
with open(os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{subsample_seed}_stats.pkl'), 'rb') as f:
	subsampled_monkey_stats = pickle.load(f)
stim_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' not in k and 'RS' not in k]
spont_dataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' in k]
df_ = make_monkey_df_time(subsampled_monkey_stats, dataset_types=stim_datataset_types_)
df_spont = make_monkey_df_time(subsampled_monkey_stats, dataset_types=spont_dataset_types_)
df_spont_ =df_spont.rename(columns={'Activity Type':'Activity Type spont','EV':'EV gray screen', 'EV shuffled':'EV shuffled gray screen','Max Corr Vals':'Max Corr Val Spont','null_shuffled': 'null_shuffled spont',
                                    'Coeff Val':'Coeff Val spont'})   
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV gray screen')	

In [ ]:
# Fig Supp 11I
fontsize=6

line_color='black'

hue='EV gray screen'
# Calculate global min and max for consistent normalization
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

fig, axes=plt.subplots(ncols=2, figsize=(2.5,1.2), sharex=True, sharey=True)

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue='EV gray screen', palette=color_map, 
	s=5, hue_norm=(global_min,global_max), ax=ax, legend=False)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	# legend = ax.legend(loc=(1.05,0.3),fontsize=fontsize*0.8)
	ax.tick_params(axis='both', labelsize=fontsize, pad=1)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=1)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=1)
	ax.set_aspect('equal')
	# legend.set_title(title='EV gray screen',prop={'size':6*0.8})
	legend.get_frame().set_linewidth(0.2)
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.5])
	ax.set_yticks([0,0.5])

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.75)
	# cbar.set_label('EV gray\nscreen', fontsize=fontsize*.8,
    #             	y=1.25,ha='right', rotation=0)  # Customize label as needed
	cbar.ax.set_title('EV gray\nscreen', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.set_yticks([0.2,0.4])
	cbar.ax.tick_params(labelsize=4, pad=0.02)
	cbar.ax.spines[:].set_linewidth(0.5)
	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	cbar.ax.set_position([pos.x0, pos.y0 -0.08, pos.width, pos.height])
plt.subplots_adjust(wspace=0.14)
if save_figs is True:
    plt.savefig(fig_dir + f'fig7_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_EV_vs_EVshuffled_colored_by_EVspon.pdf', transparent=True, bbox_inches='tight')
plt.show()


### supplemental

In [ ]:

x_order=['V4', 'V1']
x ='Area'
hue='Analysis Type'
neuron_property='Value'
perm_type='paired'
perm_t=True
hierarchical=False
mouse_or_date='Date'


with open(os.path.join(project_root, 'results/fig_5', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
main_monkey_name = 'L'
subsample_monkey_name = 'D'

melted_df_monkey_subsample = pd.DataFrame()
pval_dict_monkey_subsample = {}
pval_dict_monkey_subsample['V4'] = []
pval_dict_monkey_subsample['V1'] = []


for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	spont_dataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' in k ]
	df_ = make_monkey_df_time(subsampled_monkey_stats, dataset_types=spont_dataset_types_)
	df = df_[(df_['SNR']>2)&(df_['split-half r']>0.8)]
	melted_df = df.melt(id_vars=['Date', 'Area','Activity Type'], 
                    value_vars=['EV', 'EV shuffled', 'control shuffle EV'], 
                    var_name='Analysis Type', 
                    value_name='Value')
	melted_df['Subsample Seed'] = seed
	melted_df_monkey_subsample = pd.concat([melted_df_monkey_subsample, melted_df], ignore_index=True)
	df_stats = melted_df[melted_df['Analysis Type'].isin(['EV', 'EV shuffled'])]
	p_val1, stars1 = get_comparison_test_stars(df_stats[df_stats[x]==x_order[0]], hue, neuron_property, perm_t=perm_t, perm_type=perm_type, 
                                            hierarchical=hierarchical, mouse_or_date=mouse_or_date, central_tendency='median',
                                        	return_pval=True)
	p_val2, stars2 = get_comparison_test_stars(df_stats[df_stats[x]==x_order[1]], hue, neuron_property, perm_t=perm_t, perm_type=perm_type,
                                            hierarchical=hierarchical, mouse_or_date=mouse_or_date, central_tendency='median',
                                            return_pval=True)
	pval_dict_monkey_subsample[x_order[0]].append(p_val1)
	pval_dict_monkey_subsample[x_order[1]].append(p_val2)
	

In [ ]:
# Fig Supp 12C right
fontsize=6
areas = ['V4','V1']
fig, ax = plt.subplots(figsize=(1,1.2))
sns.violinplot(data=melted_df_monkey_subsample, x='Area', y='Value', 
            hue='Analysis Type', order=['V4', 'V1'],hue_order=['EV', 'EV shuffled'],
            linewidth=0, gap=-0.2, width=0.8, inner='box', palette=['gray','lightgray'],
            inner_kws={'box_width':2, 'alpha':0.7},
            saturation=1,cut=0)
legend = ax.legend(fontsize=fontsize*0.8, loc=(0.55,0.9))
legend.get_frame().set_linewidth(0.2)
xtick_labels = [label.get_text() for label in ax.get_xticklabels()]
ax.set_xticks(ticks=ax.get_xticks(), labels=['V1→V4', 'V4→V1'], fontsize=6)
ax.set_ylabel('EV fraction', fontsize=fontsize, labelpad=0)
ax.set(xlabel=None)
ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, size=2)
sns.despine()
custom_colors = ['#72BEB7','#B6E3DF','#EDAEAE', '#f6d6d6']
violins  = [s for s in ax.get_children() if isinstance(s, collections.PolyCollection)]
for violin, color in zip(violins, custom_colors):
    violin.set_facecolor(color)
ax.spines[:].set_linewidth(0.5)

plot_null_line(df, neuron_property='control shuffle EV', ax=ax)
# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=1.15)
palette = ['#72BEB7','#EDAEAE']
color_label(ax, palette,fontsize, predictor_color='black', y_offset=-0.075)
# ax.set_yticks(ticks=[0,0.5])
ax.set_ylim(top=0.49, bottom=-0.01)
ax.set_yticklabels([])
ax.set_ylabel(None)
ax.legend_.remove()

for x_i in x_order:
	y = melted_df_monkey_subsample[melted_df_monkey_subsample[x]==x_i][neuron_property].max()*0.8
	x_index = x_order.index(x_i)
	p_vals = pval_dict_monkey_subsample[x_i]
	median_p_val = np.median(p_vals)
	if median_p_val < 0.05:
		if median_p_val < 0.001:
			stars = '***'
		elif median_p_val < 0.01:
			stars = '**'
		else:
			stars = '*'
		ax.text(x_index, y, stars, ha='center', va='bottom', fontsize=fontsize)
	else:
		ax.text(x_index, y, 'n.s.', ha='center', va='bottom', fontsize=fontsize*0.8, color='gray')

if save_figs is True:
	plt.savefig(fig_dir + f'fig7_supp_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_EV_comparison_spont.pdf', transparent=True, bbox_inches='tight')


In [ ]:
import random 
random.seed(17)
subsample_seed = random.choice(subsample_seeds)
with open(os.path.join(project_root,f'results/fig_5/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{subsample_seed}_stats.pkl'), 'rb') as f:
	subsampled_monkey_stats = pickle.load(f)
stim_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' not in k and 'RS' not in k]
spont_dataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas'] and 'spont' in k]
df = make_monkey_df_time(subsampled_monkey_stats, dataset_types=stim_datataset_types_)
df_spont_ = make_monkey_df_time(subsampled_monkey_stats, dataset_types=spont_dataset_types_)
df_ =df.rename(columns={'Activity Type':'Activity Type stimulus','EV':'EV stimulus', 'EV shuffled':'EV shuffled stimulus','Max Corr Vals':'Max Corr Val stim','null_shuffled': 'null_shuffled stim',
'Coeff Val':'Coeff Val stim'})               
df_both = pd.merge(df_, df_spont_, on=['Date', 'Area', 'SNR', 'split-half r','1-vs-rest r²'])
df_both_sorted = df_both.sort_values('EV stimulus')

In [ ]:
# Fig Supp 12G bottom left and right
# combined figure for supp
fontsize=6

line_color='black'
fig, axes=plt.subplots(figsize=(2,1.1), ncols=2, sharex=True, sharey=True)

hue='EV stimulus'
global_min = df_both_sorted[hue].min()
global_max = df_both_sorted[hue].max()+0.05

for ax, area,color_map  in zip(axes, ['V4','V1'], [v4_cmap, v1_cmap]):
	sns.scatterplot(data=df_both_sorted[df_both_sorted.Area==area],
	x='EV', y='EV shuffled', hue=hue, palette=color_map, s=5, 
 	legend=False, ax=ax)
	ev_max = df_both_sorted[df_both_sorted.Area==area]['EV'].max()
	max_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].max()
	ev_min = df_both_sorted[df_both_sorted.Area==area]['EV'].min()
	min_val = df_both_sorted[df_both_sorted.Area==area]['EV shuffled'].min()
	ax.tick_params(axis='both', labelsize=fontsize, pad=1, width=0.3, size=2)
	ax.set_ylabel('EV shuffled', fontsize=fontsize, labelpad=0)
	ax.set_xlabel('EV unshuffled', fontsize=fontsize, labelpad=0)
	ax.set_aspect('equal')
	ax.plot([0, ev_max], [0,ev_max],color=line_color, linestyle='--', linewidth=0.8)
	sns.despine()
	ax.spines[:].set_linewidth(0.5)
	ax.set_xticks([0,0.4])
	ax.set_yticks([0,0.4])
	# ax.set_title('gray screen', fontsize=fontsize, x=0.5, y=0.99)

	# Create a colorbar with the custom colormap
	norm = Normalize(global_min,global_max)
	sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
	sm.set_array([])  # Required for matplotlib to recognize the mappable object
	cbar = plt.colorbar(sm, ax=ax, pad=0.02, shrink=0.75)
	cbar.ax.set_title('EV\nstimulus', fontsize=fontsize*0.8, pad=0.3, y=1.05, x=1.5)
	cbar.ax.tick_params(labelsize=4, pad=0.02, width=0.3, size=2)
	cbar.ax.spines[:].set_linewidth(0.5)

	# Adjust the colorbar's vertical position
	# Get the original position of the colorbar
	pos = cbar.ax.get_position()
	ax.set_xlim(left=-0.02, right=0.41)
	cbar.ax.set_position([pos.x0+0.02, pos.y0 - 0.02, pos.width, pos.height])

if save_figs is True:
	plt.savefig(fig_dir + f'fig7_supp_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_EV_vs_EVshuffled_colored_by_EVstim.pdf', transparent=True, bbox_inches='tight')
# plt.show()
